In [1]:
import math
import numpy as np

import matplotlib
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Effective Growth & Gain Of Pangea Swap

#### Reference

1. [understand-lp-risks-use-effective-gain-loss-not-impermanent-loss](https://medium.com/@gui.lamacie/understand-lp-risks-use-effective-gain-loss-not-impermanent-loss-34d1450231a5)
2. [haydenzadams](https://twitter.com/haydenzadams/status/1374795486398459908?s=20&t=PFKqXTF8aP31pvFkTuZcQQ)

## 유동성을 공급하는 경우, 해당 포지션의 자산 가치

공급된 유동성에 대해서 어떻게 가치를 평가할 수 있을까요? 이는 해당 포지션을 소각하였을 때, 얼마만큼의 토큰으로 반환되는지를 통해 역산해볼 수 있습니다.

* $X$ : 포지션에 예치된 실제 X 토큰의 갯수
* $Y$ : 포지션에 예치된 실제 Y 토큰의 갯수
* $P_c$ : 현재 가격 ($P = \frac{Y}{X}$)
* $V$ : 포지션의 총 자산가치

일 때

$$
V = X\cdot P_c + Y
$$

입니다. 

우리는 LPing을 하였을 때, 중요하게 바라봐야 하는 것은 **포지션의 총 자산가치**에 있습니다. 

> DEX에서의 LP을 공급한다는 것, 즉 LP 포지션을 갖는다는 것은 **포지션의 총 자산가치 손실 리스크**을 통해, **거래 수익**을 발생시키는 **금융 상품**입니다.

집중화된 유동성 모델의 **혁신**은 포지션의 **가격 범위** 설정으로 **총 자산가치 손실 리스크와 거래 수익을 관리**할 수 있다는 점입니다. 더 나아가 예치자는 두 토큰의 가격 변화를 예측하여, 집중화된 유동성은 총 자산가치 손실 리스크를 줄이면서, 거래 수익을 끌어올릴 수 있는 전략도 가능합니다.

## 가격 변화에 따른 포지션 내 자산 변동

![image.png](https://imgur.com/cm4fm89.png)

가격이 변할 때, 포지션에 있는 자산은 위와 같이 변화하게 됩니다.  가격이 떨어지는 경우에는 X의 비중이 높아지고, 가격이 높아지는 경우에는 Y의 비중이 높아지는 방향으로 움직입니다.

유니스왑 V3가 유니스왑 V2에 비해 가격 변화에 따라 자산이 더 크게 변동하기 때문에, **더 많은 손실을 초래한다** 라고 생각하기 쉽습니다. 이는 **실질적 자산 변동**, 즉 EGL(Effective Gain & Loss)을 기준으로 바라보면 그렇지 않다는 것을 알 수 있습니다. 

### Effective Gain & Loss, 실질 변동액

$$
EGL = V_{after} - V_{before} + Fee
$$

EGL은 자산 가치 총액 변동과 수수료의 합산으로, EGL이 0보다 크다는 것은 유저는 LPing을 한 후, 자산가치가 더 높아졌다는 것을 의미하고, EGL이 0보다 작다는 것은 유저는 LPing을 한 후 자산가치가 더 낮아졌다는 것을 의미합니다.

현재 가격이 변동하는 경우 Uniswap V2 대비 Uniswap V3가 어떻게 동작하는지 보도록 하겠습니다.

#### 예제 : USDT / ETH 풀에 유동성을 공급 : Full Range 포지션 VS 집중된 포지션

현재 시장에서의 거래 가격이 2,000 USDT = 1 ETH 라고 합시다. LPing을 한 후, 이더리움의 가격이 10%가 올랐을 떄, 10%가 내렸을 떄 포지션의 가치가 어떻게 변화하는지 **EGL**을 통해 파악해보도록 하겠습니다. 


![](https://imgur.com/0PgmJva.png)

### 1. 두 포지션의 유동성 크기 비교


우선 비교를 편하게 하기 위해, Concentrated / Full Range 모두 2000 DAI와 1 ETH 씩 예치한 상황이라고 봅시다. 그렇다면 현재 유동성은 얼마일까요?? 

$
L_{concentrated} = \frac{Y}{\sqrt{P_c} - \sqrt{P_l}} = \frac{2000}{\sqrt{2000} - \sqrt{1000}} = 152.69
$

$
L_{full range} = \frac{Y}{\sqrt{P_c} - \sqrt{P_l}} = \frac{2000}{\sqrt{2000} - \sqrt{0}} = 44.47
$

*c.f) X를 기준으로 계산하더라도, 유동성 크기는 동일합니다.*

In [2]:
liquidity_conc = 2000 / (math.sqrt(2000) - math.sqrt(1000))
liquidity_full = 2000 / (math.sqrt(2000) - 0)

print(f"집중화된 포지션 : {liquidity_conc:.3f}")
print(f"전체    포지션 : {liquidity_full:.3f}")
print(f"Fee Multiplier : {liquidity_conc / liquidity_full:.2f}")

집중화된 포지션 : 152.688
전체    포지션 : 44.721
Fee Multiplier : 3.41


동일한 자산을 넣었을 때, 50% ~ 200% 범위로 좁혀서 넣은 집중화된 포지션과 전체 범위로 넣은 전체 포지션의 유동성은 대략 **3.41**배 차이가 발생합니다. 즉 즉 전체 포지션으로 공급했을 때 보다 3.41배의 거래를 성사시킬 수 있고, 이로 인해 수수료 수익도 3.41배 수준으로 벌어들일 수 있다는 것을 의미합니다.

### 2. 가격이 10% 상승 or 하락 했을 때, 두 포지션의 자산 가치 비교

포지션에 예치되어 있는 토큰 갯수는 아래와 같이 생각하면 됩니다. 즉 포지션을 해제했을 때, 받을 수 있는 토큰 갯수가 됩니다.

$
X = (\frac{1}{\sqrt{P_c}} - \frac{1}{\sqrt{P_h}})L \\
Y = (\sqrt{P_c} - \sqrt{P_l})L
$

집중화된 포지션에 예치했을 때와 전체 포지션에 예치했을 때 각각 차이는 아래와 같습니다.

In [3]:
price0 = 1800
X_conc0 = liquidity_conc * (1/math.sqrt(price0) - 1/math.sqrt(4000))
Y_conc0 = liquidity_conc * (math.sqrt(price0) - math.sqrt(1000))

X_full0 = liquidity_full * (1/math.sqrt(price0))
Y_full0 = liquidity_full * (math.sqrt(price0))

price1 = 2200
X_conc1 = liquidity_conc * (1/math.sqrt(price1) - 1/math.sqrt(4000))
Y_conc1 = liquidity_conc * (math.sqrt(price1) - math.sqrt(1000))

X_full1 = liquidity_full * (1/math.sqrt(price1))
Y_full1 = liquidity_full * (math.sqrt(price1))

이 경우 실질적인 자산 가치 손실은 어떻게 될까요?? 10% 가격이 하락했을 때와 10% 가격이 상승했을 때 모두 비교해보겠습니다.


In [4]:
# 초기 자산 가치
asset_original = 1 * 2000 + 2000

# 가격 하락 후 포지션의 자산 가치
asset_conc0 = X_conc0 * price0 + Y_conc0
asset_full0 = X_full0 * price0 + Y_full0

# 가격 상승 후 포지션의 자산 가치
asset_conc1 = X_conc1 * price1 + Y_conc1
asset_full1 = X_full1 * price1 + Y_full1


print("case 1) 가격 하락 시 자산 가치 변동")
print(f"집중화된 포지션의 자산 가치 변동 = {(asset_conc0 - asset_original) / asset_original * 100:.2f}%")
print(f"전체    포지션의 자산 가치 변동 = {(asset_full0 - asset_original) / asset_original * 100:.2f}%")


print("")
print("case 2) 가격 상승 시 자산 가치 변동")
print(f"집중화된 포지션의 자산 가치 변동 = {(asset_conc1 - asset_original) / asset_original * 100:.2f}%")
print(f"전체    포지션의 자산 가치 변동 = {(asset_full1 - asset_original) / asset_original * 100:.2f}%")

case 1) 가격 하락 시 자산 가치 변동
집중화된 포지션의 자산 가치 변동 = -5.45%
전체    포지션의 자산 가치 변동 = -5.13%

case 2) 가격 상승 시 자산 가치 변동
집중화된 포지션의 자산 가치 변동 = 4.59%
전체    포지션의 자산 가치 변동 = 4.88%


즉 **집중화된 유동성으로 공급했을 경우**, 가격 변동 시 총 자산 가치는 -0.3% 수준으로 떨어지지만, 그로 인해 벌어들일 수 있는 Fee 수익은 **3.41배가 증가**합니다.

집중화된 유동성은 포지셔닝을 어떻게 두냐에 따라 달라질 수 있습니다.

## 가격 범위를 비대칭으로 두는 포지셔닝 전략

현재 가격을 기준으로 꼭 가격 범위를 동일하게 두지 않아도 됩니다. 핵심은 풀을 구성하는 두 토큰의 **상대 가격**이 이후 어떻게 변화할 것으로 바라보느냐에 따라 LP 포지션을 서로 다르게 가져갈 수 있고, 이에 따라 자산 손실을 효과적으로 대처할 수 있습니다. 

In [5]:
def value_difference(k, pos_lower=0.5, pos_higher=2.0):
    """포지션의 가격 범위에 따라 자산가치 변동율 계산
    @params k 가격 변동 수준 계수, (1.1이면 10% 가격 상승, 0.9이면 10% 가격 하락)
    @params pos_lower 포지션의 하방 가격선
    @params pos_upper 포지션의 상방 가격선
    """
    
    def calculate_value(p_c, p_l, p_h):
        """
        @param p_c 현재 가격
        @param p_l 포지션의 하방 가격선
        @param p_h 포지션의 상방 가격선
        """
        # 범위에 따른 자산 가치 = (x 갯수 * price + y 갯수)
        if (p_c < p_h):
            X = (1/math.sqrt(p_c) - 1/math.sqrt(p_h))
        else:
            # 현재 가격이 포지션 범위를 초과할 경우, X의 갯수는 0 
            X = 0        
            
        if (p_c > p_l):
            Y = (math.sqrt(p_c) - math.sqrt(p_l))
        else:
            # 현재 가격이 포지션 범위 밑으로 내려갈 경우, Y의 갯수는 0
            Y = 0
        return p_c * X + Y
    
    init_value = calculate_value(1.0, pos_lower, pos_higher)
    curr_value = calculate_value(k, pos_lower, pos_higher)
    
    return (curr_value - init_value) / init_value

In [6]:
def il(k, pos_lower=0.5, pos_higher=2.0):
    """포지션의 가격 범위에 따라 자산가치 변동율 계산
    @params k 가격 변동 수준 계수, (1.1이면 10% 가격 상승, 0.9이면 10% 가격 하락)
    @params pos_lower 포지션의 하방 가격선
    @params pos_upper 포지션의 상방 가격선
    """
    
    def calculate_value(p_c, p_l, p_h):
        """
        @param p_c 현재 가격
        @param p_l 포지션의 하방 가격선
        @param p_h 포지션의 상방 가격선
        """
        # 범위에 따른 자산 가치 = (x 갯수 * price + y 갯수)
        if (p_c < p_h):
            X = (1/math.sqrt(max(p_c,p_l)) - 1/math.sqrt(p_h))
        else:
            # 현재 가격이 포지션 범위를 초과할 경우, X의 갯수는 0 
            X = 0        
            
        if (p_c > p_l):
            Y = (math.sqrt(min(p_c,p_h)) - math.sqrt(p_l))
        else:       
            # 현재 가격이 포지션 범위 밑으로 내려갈 경우, Y의 갯수는 0
            Y = 0
        return p_c * X + Y
    
    def calculate_origin(k, p_c, p_l, p_h):
        # 범위에 따른 자산 가치 = (x 갯수 * price + y 갯수)
        if (p_c < p_h):
            X = (1/math.sqrt(max(p_c,p_l)) - 1/math.sqrt(p_h))
        else:
            X = 0        
            
        if (p_c > p_l):
            Y = (math.sqrt(min(p_c,p_h)) - math.sqrt(p_l))
        else:       
            # 현재 가격이 포지션 범위 밑으로 내려갈 경우, Y의 갯수는 0
            Y = 0
        return k * X + Y    

    init_value = calculate_origin(k, 1.0, pos_lower, pos_higher)
    curr_value = calculate_value(k, pos_lower, pos_higher)
    
    return (curr_value - init_value) / init_value

In [7]:
p_l = 0.999
p_h = 2.0

print("가격 변동에 따른 포지션 별 차이")

p_t = 0.9
print("10% 가격 하락 시")
print(f"50% ~ 200%으로 공급한 경우 : {value_difference(p_t,  p_l,    p_h) * 100:.2f}%")
print(f"0% ~   ∞%으로 공급한 경우  : {value_difference(p_t,    0, np.inf) * 100:.2f}%")
print(f"0% ~ 200%으로 공급한 경우  : {value_difference(p_t,    0,    p_h) * 100:.2f}%")
print(f"50% ~  ∞%으로 공급한 경우  : {value_difference(p_t,  p_l, np.inf) * 100:.2f}%")


print("\n")

p_t = 1.1
print("10% 가격 상승 시")
print(f"50% ~ 200%으로 공급한 경우 : {value_difference(p_t,   p_l,    p_h) * 100:.2f}%")
print(f"0% ~   ∞%으로 공급한 경우  : {value_difference(p_t,     0, np.inf) * 100:.2f}%")
print(f"0% ~ 200%으로 공급한 경우  : {value_difference(p_t,     0,    p_h) * 100:.2f}%")
print(f"50% ~  ∞%으로 공급한 경우  : {value_difference(p_t,   p_l, np.inf) * 100:.2f}%")

가격 변동에 따른 포지션 별 차이
10% 가격 하락 시
50% ~ 200%으로 공급한 경우 : 6.44%
0% ~   ∞%으로 공급한 경우  : -5.13%
0% ~ 200%으로 공급한 경우  : -2.47%
50% ~  ∞%으로 공급한 경우  : -5.18%


10% 가격 상승 시
50% ~ 200%으로 공급한 경우 : 9.17%
0% ~   ∞%으로 공급한 경우  : 4.88%
0% ~ 200%으로 공급한 경우  : 2.08%
50% ~  ∞%으로 공급한 경우  : 9.76%


![](https://imgur.com/sw8tOAl.png)

포지셔닝 전략에 따라, 유저는 기존 V2에 대비하여 보다 **보수적으로도** 자산을 운영하거나, **공격적으로** 자산을 배치할 수 있습니다. 특히 **Stable - Volatile** 페어에 투자하는 경우에 이를 바탕으로 사고하면 훨씬 더 리스크 관리와 수익성을 극대화할 수 있습니다. 



**포지셔닝 전략 핵심**

* 가격이 하락할 때에는 가격이 낮은 방향으로 범위를 벌려둘 경우 자산 손실 줄일 수 있음 (대신 가격이 올라갈 경우 이익률이 떨어짐)
* 가격이 올라갈 때에는 가격이 높은 방향으로 범위를 벌려둘 경우 자산 이익을 늘어남 (대신 가격이 떨어질 경우 손실이 커짐 )

### 요약

![](https://imgur.com/zlqlw3M.png)